In [1]:
#one time install of torch, transformers
!pip install torch
!pip install transformers

In [2]:
import numpy as np
import pandas as pd
import torch
import transformers as ppb # pytorch transformers
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import *
import datetime

In [3]:
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

In [37]:
VSafe_symptoms_records_df = pd.read_csv(r'C:\Users\badri.prasad\OneDrive - GDIT\Documents\VSAFE\VSafeCoded_records_until_11_1420221206094319\VSafe-Coding-symptoms-description-Coded-Records-11-14.csv')

In [5]:
VSafe_symptoms_records_df.head()

,OtherTypeFileName,Symptoms_Other,ResponseOtherNbr,ResponseID,Survey_Static_ID,Registrant_Code,Started_On,DS_Name,ptEnglish,lltEnglish,lltVersion
0,symptoms_description,Hematoma at injection site. Appeared last week.,74650,s243977125742181582231051202904933953463,vsafe-14-day-dose1,3SP-26919-22331,12/28/2020,vsafe_surveys_20201214-20210112.csv,Injection site haematoma,Hematoma injection site,25.1
1,symptoms_description,Persistent headache and stiff neck,74651,s243998780637162849260341594616872193448,vsafe-14-day-dose1,3V2-32861-01404,12/28/2020,vsafe_surveys_20201214-20210112.csv,Headache,Headache,25.1
2,symptoms_description,Persistent headache and stiff neck,74651,s243998780637162849260341594616872193448,vsafe-14-day-dose1,3V2-32861-01404,12/28/2020,vsafe_surveys_20201214-20210112.csv,Musculoskeletal stiffness,Stiff neck,25.1
3,symptoms_description,Congestion. Sore throat. Ear ache.,74652,s243985592580555985369977218646562988981,vsafe-14-day-dose1,4ZK-13405-96024,12/28/2020,vsafe_surveys_20201214-20210112.csv,Ear pain,Ear ache,25.1
4,symptoms_description,Congestion. Sore throat. Ear ache.,74652,s243985592580555985369977218646562988981,vsafe-14-day-dose1,4ZK-13405-96024,12/28/2020,vsafe_surveys_20201214-20210112.csv,Respiratory tract congestion,Respiratory tract congestion,25.1


In [6]:
VSafe_symptoms_records_df.tail()

,OtherTypeFileName,Symptoms_Other,ResponseOtherNbr,ResponseID,Survey_Static_ID,Registrant_Code,Started_On,DS_Name,ptEnglish,lltEnglish,lltVersion
73348,symptoms_description,my body feels very week and im really drowsy. ...,349589,s245588030979798264855047341172968840688,vsafe-21-day-dose1-asz-mod,Q75-20932-01742,01/13/2021,vsafe_surveys_20201214-20210112.csv,Asthenia,Weakness,25.1
73349,symptoms_description,"Sore throat , runny nose , loss of taste and s...",349590,s245604809383298450861596951302227257052,vsafe-21-day-dose1-nvx-pfr,LGS-24181-04569,01/13/2021,vsafe_surveys_20201214-20210112.csv,Anosmia,Loss of smell,25.1
73350,symptoms_description,"Sore throat , runny nose , loss of taste and s...",349590,s245604809383298450861596951302227257052,vsafe-21-day-dose1-nvx-pfr,LGS-24181-04569,01/13/2021,vsafe_surveys_20201214-20210112.csv,Ageusia,Loss of taste,25.1
73351,symptoms_description,"Sore throat , runny nose , loss of taste and s...",349590,s245604809383298450861596951302227257052,vsafe-21-day-dose1-nvx-pfr,LGS-24181-04569,01/13/2021,vsafe_surveys_20201214-20210112.csv,Rhinorrhoea,Runny nose,25.1
73352,symptoms_description,"Sore throat , runny nose , loss of taste and s...",349590,s245604809383298450861596951302227257052,vsafe-21-day-dose1-nvx-pfr,LGS-24181-04569,01/13/2021,vsafe_surveys_20201214-20210112.csv,Oropharyngeal pain,Sore throat,25.1


In [7]:
VSafe_symptoms_records_df['Symptoms_Other'] = VSafe_symptoms_records_df['Symptoms_Other'].fillna('BLANK')

In [8]:
VSafe_symptoms_records_df_lltEnglish_value_counts = VSafe_symptoms_records_df['lltEnglish'].value_counts()

In [9]:
VSafe_symptoms_records_df_lltEnglish_value_counts.head(15)

Headache                        7234
Fatigue                         3772
Sore throat                     2689
Cough                           2639
Nausea                          1847
Injection site redness          1826
Generalized aching              1780
Fever                           1684
Diarrhea                        1591
Chills                          1546
Respiratory tract congestion    1522
Injection site itching          1477
Runny nose                      1371
COVID-19                        1326
COVID-19 virus test positive    1279
Name: lltEnglish, dtype: int64

In [10]:
VSafe_symptoms_records_df['ResponseOtherNbr'].nunique()

32813

In [11]:
VSafe_symptoms_records_df[VSafe_symptoms_records_df['ResponseOtherNbr'] == 74651]

,OtherTypeFileName,Symptoms_Other,ResponseOtherNbr,ResponseID,Survey_Static_ID,Registrant_Code,Started_On,DS_Name,ptEnglish,lltEnglish,lltVersion
1,symptoms_description,Persistent headache and stiff neck,74651,s243998780637162849260341594616872193448,vsafe-14-day-dose1,3V2-32861-01404,12/28/2020,vsafe_surveys_20201214-20210112.csv,Headache,Headache,25.1
2,symptoms_description,Persistent headache and stiff neck,74651,s243998780637162849260341594616872193448,vsafe-14-day-dose1,3V2-32861-01404,12/28/2020,vsafe_surveys_20201214-20210112.csv,Musculoskeletal stiffness,Stiff neck,25.1


In [12]:
VSafe_symptoms_records_df_cleaned = pd.DataFrame()

VSafe_symptoms_records_df_cleaned['ID_txt_zip'] = pd.Series(zip(VSafe_symptoms_records_df['ResponseOtherNbr'], VSafe_symptoms_records_df['Symptoms_Other'])).unique()
VSafe_symptoms_records_df_cleaned['Resp-ID'] = VSafe_symptoms_records_df_cleaned['ID_txt_zip'].map(lambda x: x[0])
VSafe_symptoms_records_df_cleaned['Symptom_txt'] = VSafe_symptoms_records_df_cleaned['ID_txt_zip'].map(lambda x: x[1])
VSafe_symptoms_records_df_cleaned = VSafe_symptoms_records_df_cleaned.drop('ID_txt_zip', axis = 1)

for LLT in VSafe_symptoms_records_df_lltEnglish_value_counts.head(15).index:
    VSafe_symptoms_records_df_cleaned[LLT] = 0
    
for ID in VSafe_symptoms_records_df_cleaned['Resp-ID'].unique():
    for LLT in VSafe_symptoms_records_df[VSafe_symptoms_records_df['ResponseOtherNbr'] == ID]['lltEnglish'].unique():
        if LLT in VSafe_symptoms_records_df_lltEnglish_value_counts.head(15).index:
            VSafe_symptoms_records_df_cleaned.loc[VSafe_symptoms_records_df_cleaned['Resp-ID'] == ID, LLT] = 1


In [13]:
VSafe_symptoms_records_df_cleaned.shape

(32813, 17)

In [14]:
VSafe_symptoms_records_df_cleaned[VSafe_symptoms_records_df_cleaned['Symptom_txt'].isna()]

,Resp-ID,Symptom_txt,Headache,Fatigue,Sore throat,Cough,Nausea,Injection site redness,Generalized aching,Fever,Diarrhea,Chills,Respiratory tract congestion,Injection site itching,Runny nose,COVID-19,COVID-19 virus test positive


In [15]:
VSafe_symptoms_records_df_cleaned[VSafe_symptoms_records_df_cleaned['Symptom_txt'] == 'BLANK']

,Resp-ID,Symptom_txt,Headache,Fatigue,Sore throat,Cough,Nausea,Injection site redness,Generalized aching,Fever,Diarrhea,Chills,Respiratory tract congestion,Injection site itching,Runny nose,COVID-19,COVID-19 virus test positive
6985,163823,BLANK,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7924,176235,BLANK,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
17489,213740,BLANK,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
20730,259868,BLANK,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
31594,346365,BLANK,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
32744,347523,BLANK,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [16]:
VSafe_symptoms_records_df_lltEnglish_value_counts_1000_up = VSafe_symptoms_records_df_lltEnglish_value_counts[VSafe_symptoms_records_df_lltEnglish_value_counts > 1000].index

In [17]:
VSafe_symptoms_records_df_rel = VSafe_symptoms_records_df[VSafe_symptoms_records_df['lltEnglish'].isin(VSafe_symptoms_records_df_lltEnglish_value_counts_1000_up)]

In [18]:
#VSafe_symptoms_records_df_rel[['Symptoms_Other', 'ResponseID', 'lltEnglish']].to_csv(r'C:\Users\badri.prasad\OneDrive - GDIT\Documents\VSAFE\VSafeCoded_records_until_11_1420221206094319\candidate_LLT.csv')

In [19]:
# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [20]:
tokenized = VSafe_symptoms_records_df_cleaned['Symptom_txt'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

In [21]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])  

In [22]:
np.array(padded).shape

(32813, 88)

In [23]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(32813, 88)

In [24]:
padded = padded
attention_mask = attention_mask

In [25]:
last_hidden_states_list = []

for i in range(int(np.ceil(padded.shape[0]/2000))):
    input_ids_iter = torch.tensor(padded[(2000*i):(2000*(i + 1))])  
    attention_mask_iter = torch.tensor(attention_mask[(2000*i):(2000*(i + 1))])
    
    with torch.no_grad():
        last_hidden_states = model(input_ids_iter, attention_mask=attention_mask_iter)
        
    last_hidden_states_list.append(last_hidden_states.last_hidden_state)
    
    print('Iteration ' + str((i + 1)) + ' is completed at ' + str(datetime.datetime.now()))

Iteration 1 is completed at 2023-02-09 15:25:21.255249
Iteration 2 is completed at 2023-02-09 15:26:11.327765
Iteration 3 is completed at 2023-02-09 15:26:59.204615
Iteration 4 is completed at 2023-02-09 15:27:48.491742
Iteration 5 is completed at 2023-02-09 15:28:36.275562
Iteration 6 is completed at 2023-02-09 15:29:23.061494
Iteration 7 is completed at 2023-02-09 15:30:11.204896
Iteration 8 is completed at 2023-02-09 15:31:00.240296
Iteration 9 is completed at 2023-02-09 15:31:47.556472
Iteration 10 is completed at 2023-02-09 15:32:35.368981
Iteration 11 is completed at 2023-02-09 15:33:22.846613
Iteration 12 is completed at 2023-02-09 15:34:10.576618
Iteration 13 is completed at 2023-02-09 15:34:59.134953
Iteration 14 is completed at 2023-02-09 15:35:47.978369
Iteration 15 is completed at 2023-02-09 15:36:36.453984
Iteration 16 is completed at 2023-02-09 15:37:24.289742
Iteration 17 is completed at 2023-02-09 15:37:43.993736


In [26]:
last_hidden_states_cat = torch.cat(tuple(last_hidden_states_list), 0)

In [27]:
features = last_hidden_states_cat[:,0,:].numpy()

In [28]:
#lr_clf = LogisticRegression(max_iter = 3000)
#lr_clf.fit(train_features, train_labels)     

In [29]:
from sklearn.metrics import *

In [30]:
dataframe_data_list = []

for LLT in VSafe_symptoms_records_df_lltEnglish_value_counts_1000_up:
    train_features, test_features, train_labels, test_labels = train_test_split(features, VSafe_symptoms_records_df_cleaned[LLT])
    lr_clf = LogisticRegression(max_iter=3000)
    lr_clf.fit(train_features, train_labels)
    
    fpr, tpr, thresholds = roc_curve(test_labels, lr_clf.predict(test_features), pos_label=1)

    
    print('For ' + LLT + ', The efficacy of the model is (out of 1) ' + str(lr_clf.score(test_features, test_labels)))
    
    print('For ' + LLT + ', Accuracy is ' + str(accuracy_score(test_labels, lr_clf.predict(test_features))))
    print('For ' + LLT + ', Recall is ' + str(recall_score(test_labels, lr_clf.predict(test_features))))
    print('For ' + LLT + ', Precision is ' + str(precision_score(test_labels, lr_clf.predict(test_features))))
    print('For ' + LLT + ', F1-Score is ' + str(f1_score(test_labels, lr_clf.predict(test_features))))
    print('For ' + LLT + ', AUC is ' + str(auc(fpr, tpr)))#
    
    dataframe_data_list.append([LLT, round(accuracy_score(test_labels, lr_clf.predict(test_features)), 4), round(recall_score(test_labels, lr_clf.predict(test_features)), 4),
                               round(precision_score(test_labels, lr_clf.predict(test_features)), 4), round(f1_score(test_labels, lr_clf.predict(test_features)), 4),
                               round(auc(fpr, tpr), 4)])

For Headache, The efficacy of the model is (out of 1) 0.9539249146757679
For Headache, Accuracy is 0.9539249146757679
For Headache, Recall is 0.8942307692307693
For Headache, Precision is 0.8922121896162528
For Headache, F1-Score is 0.8932203389830509
For Headache, AUC is 0.9322769756657264
For Fatigue, The efficacy of the model is (out of 1) 0.977206240858118
For Fatigue, Accuracy is 0.977206240858118
For Fatigue, Recall is 0.8701007838745801
For Fatigue, Precision is 0.9162735849056604
For Fatigue, F1-Score is 0.8925904652498565
For Fatigue, AUC is 0.9301946950422005
For Sore throat, The efficacy of the model is (out of 1) 0.9842759629449049
For Sore throat, Accuracy is 0.9842759629449049
For Sore throat, Recall is 0.8583941605839416
For Sore throat, Precision is 0.9483870967741935
For Sore throat, F1-Score is 0.9011494252873562
For Sore throat, AUC is 0.9270691377464195
For Cough, The efficacy of the model is (out of 1) 0.9790346172598733
For Cough, Accuracy is 0.9790346172598733
Fo

In [31]:
# Step to write out the data
pd.DataFrame(dataframe_data_list, columns = ['LLT', 'ACCURACY', 'RECALL', 'PRECISION', 'F1-SCORE', 'AUC']).to_csv('Test-15.csv')

In [31]:
dataframe_data_list

[['Headache', 0.9539, 0.8942, 0.8922, 0.8932, 0.9323],
 ['Fatigue', 0.9772, 0.8701, 0.9163, 0.8926, 0.9302],
 ['Sore throat', 0.9843, 0.8584, 0.9484, 0.9011, 0.9271],
 ['Cough', 0.979, 0.843, 0.8818, 0.862, 0.9167],
 ['Nausea', 0.9867, 0.8184, 0.9288, 0.8701, 0.9074],
 ['Injection site redness', 0.9773, 0.7609, 0.8216, 0.7901, 0.8755],
 ['Generalized aching', 0.9737, 0.6524, 0.7965, 0.7173, 0.8217],
 ['Fever', 0.9842, 0.7743, 0.8961, 0.8307, 0.8848],
 ['Diarrhea', 0.9885, 0.7876, 0.962, 0.8661, 0.893],
 ['Chills', 0.9872, 0.8251, 0.9079, 0.8645, 0.9104],
 ['Respiratory tract congestion', 0.9822, 0.7637, 0.8575, 0.8079, 0.8786],
 ['Injection site itching', 0.9795, 0.7134, 0.7685, 0.7399, 0.8521],
 ['Runny nose', 0.9927, 0.8911, 0.9382, 0.914, 0.9442],
 ['COVID-19', 0.9749, 0.6122, 0.692, 0.6497, 0.8007],
 ['COVID-19 virus test positive', 0.9807, 0.6855, 0.7899, 0.734, 0.8391]]

In [32]:
pd.DataFrame(dataframe_data_list, columns = ['LLT', 'ACCURACY', 'RECALL', 'PRECISION', 'F1-SCORE', 'AUC'])

,LLT,ACCURACY,RECALL,PRECISION,F1-SCORE,AUC
0,Headache,0.9539,0.8942,0.8922,0.8932,0.9323
1,Fatigue,0.9772,0.8701,0.9163,0.8926,0.9302
2,Sore throat,0.9843,0.8584,0.9484,0.9011,0.9271
3,Cough,0.9790,0.8430,0.8818,0.8620,0.9167
4,Nausea,0.9867,0.8184,0.9288,0.8701,0.9074
5,Injection site redness,0.9773,0.7609,0.8216,0.7901,0.8755
6,Generalized aching,0.9737,0.6524,0.7965,0.7173,0.8217
7,Fever,0.9842,0.7743,0.8961,0.8307,0.8848
8,Diarrhea,0.9885,0.7876,0.9620,0.8661,0.8930
9,Chills,0.9872,0.8251,0.9079,0.8645,0.9104


In [34]:
import pickle

In [36]:
for LLT in VSafe_symptoms_records_df_lltEnglish_value_counts_1000_up:
    train_features, test_features, train_labels, test_labels = train_test_split(features, VSafe_symptoms_records_df_cleaned[LLT])
    lr_clf = LogisticRegression(max_iter=3000)
    lr_clf.fit(train_features, train_labels)
    
    pickle_file_name = LLT + ' model.pkl'
    
    with open('VSAFE Model Parameters\ ' + pickle_file_name ,'wb') as f:
        pickle.dump(lr_clf,f)


# Unit Testing

In [35]:
for LLT in VSafe_symptoms_records_df_lltEnglish_value_counts_1000_up[:1]:
    train_features, test_features, train_labels, test_labels = train_test_split(features, VSafe_symptoms_records_df_cleaned[LLT])
    lr_clf = LogisticRegression(max_iter=3000)
    lr_clf.fit(train_features, train_labels)
    
    fpr, tpr, thresholds = roc_curve(test_labels, lr_clf.predict(test_features), pos_label=1)

    
    print('For ' + LLT + ', The efficacy of the model is (out of 1) ' + str(lr_clf.score(test_features, test_labels)))
    
    print('For ' + LLT + ', Accuracy is ' + str(accuracy_score(test_labels, lr_clf.predict(test_features))))
    print('For ' + LLT + ', Recall is ' + str(recall_score(test_labels, lr_clf.predict(test_features))))
    print('For ' + LLT + ', Precision is ' + str(precision_score(test_labels, lr_clf.predict(test_features))))
    print('For ' + LLT + ', F1-Score is ' + str(f1_score(test_labels, lr_clf.predict(test_features))))
    print('For ' + LLT + ', AUC is ' + str(auc(fpr, tpr)))

For Headache, The efficacy of the model is (out of 1) 0.9557532910775232
For Headache, Accuracy is 0.9557532910775232
For Headache, Recall is 0.8847199564980968
For Headache, Precision is 0.9150731158605174
For Headache, F1-Score is 0.8996405861210948
For Headache, AUC is 0.9304982343370296


In [ ]:
NEXT STEPS:
    (a) Increase max_iter to eliminate error. See website
    (b) Write results to a dataframe. The results are better read in a table
    (c) Print metrics to 4 decimal places

In [24]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

In [28]:
labels = le.fit_transform(VSafe_symptoms_records_df_rel['lltEnglish'])

In [29]:
labels

array([ 8, 12, 14, ..., 14, 13, 14])

In [30]:
train_features, test_features, train_labels, test_labels = train_test_split(features, labels)

In [35]:
lr_clf = LogisticRegression()
lr_clf.fit(train_features, train_labels)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [36]:
lr_clf.score(test_features, test_labels)

0.5315626488804193

In [37]:
from sklearn.dummy import DummyClassifier
clf = DummyClassifier()

scores = cross_val_score(clf, train_features, train_labels)
print("Dummy classifier score: %0.3f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Dummy classifier score: 0.217 (+/- 0.00)


In [38]:
from sklearn.metrics import *

In [ ]:
print('Accuracy is ' + str(accuracy_score(test_labels, lr_cif.predict(test_features, test_labels))))
print('Recall is ' + str(recall_score(test_labels, lr_cif.predict(test_features, test_labels))))
print('Recall is ' + str(precision_score(test_labels, lr_cif.predict(test_features, test_labels))))
print('F1-Score is ' + str(f1_score(test_labels, lr_cif.predict(test_features, test_labels))))
print('Recall is ' + str(recall_score(test_labels, lr_cif.predict(test_features, test_labels))))